In [ ]:
import pickle, json



In [ ]:
atis_fn = "/home/yoga/workspace/knodle_hackathon_fork/knodle/hackathon_seq/atis.json"

In [ ]:
with open(atis_fn, "rb") as input_file:
    atis_dataset = json.load(input_file, encoding="UTF8")
    

In [ ]:
#with open(atis_fn, "rb") as input_file:
#    atis_dataset = pickle.load(input_file, encoding="UTF8")

In [ ]:
atis_dataset.keys()

In [ ]:
#s1 = atis_dataset['train_sents'][0]
#word_to_id = atis_dataset['vocab']
#id_to_word = {word_to_id[word]:word for word in word_to_id}
#id_to_word

In [ ]:
[id_to_word[wid] for wid in s1]

In [ ]:
l1 = atis_dataset['train_labels'][0]
#label_to_id = atis_dataset['label_dict']
#id_to_label = {label_to_id[label]:label for label in label_to_id}
#id_to_label

In [ ]:
[(id_to_word[wid], id_to_label[lid]) for (wid, lid) in zip(s1,l1)]

In [ ]:
l1

In [ ]:
# knodle/transformation/majority.py:
#    if not use_probabilistic_labels:
#        # convert labels represented as a prob distribution to a single label using majority voting
#        kwargs = {"choose_random_label": True, "other_class_id": other_class_id}
#        noisy_y_train = np.apply_along_axis(probabilies_to_majority_vote, axis=1, arr=noisy_y_train, **kwargs)


In [ ]:
data = atis_dataset

In [ ]:
import math
train_dev_sents = data["train_sents"] # list of lists
train_dev_labels = data["train_labels"] # list of lists
num_train = math.floor(0.8 * len(train_dev_sents))
train_sents = train_dev_sents[:num_train]
train_labels = train_dev_labels[:num_train]
dev_sents = train_dev_sents[num_train:]
dev_labels = train_dev_labels[num_train:]
test_sents = data["test_sents"]
test_labels = data["test_labels"]
word_to_id = data["vocab"]
label_to_id = data["label_dict"]

In [ ]:
UNK_TOKEN = "<UNK>"
PAD_TOKEN = "<PAD>"
VOCAB_SIZE = len(word_to_id)
NUM_LABELS = len(label_to_id)
EMBEDDING_SIZE = 50
HIDDEN_SIZE=50
MAX_LENGTH=20
KERNEL_SIZE=5

In [ ]:
#  We reorganize the data so that "<PAD>" has id 0, "<UNK>" has id 1, and the label 'O' has id 0 (and all words are still represented). Make sure to also change word_to_id and label_to_id.
id_to_word = {word_to_id[word]: word for word in word_to_id}
id_to_label = {label_to_id[label]: label for label in label_to_id}

def switch_idx(sents, i, j):
    D = {i:j, j:i}
    return [[D.get(word, word) for word in sent] for sent in sents]

for i, j in [(word_to_id["<PAD>"], 0), (word_to_id["<UNK>"], 1)]:
    train_sents = switch_idx(train_sents, i, j)
    dev_sents = switch_idx(dev_sents, i, j)
    test_sents = switch_idx(test_sents, i, j)
    
train_labels = switch_idx(train_labels, label_to_id["O"], 0)
dev_labels = switch_idx(dev_labels, label_to_id["O"], 0)
test_labels = switch_idx(test_labels, label_to_id["O"], 0)

word_to_id[id_to_word[0]] = word_to_id["<PAD>"]
word_to_id[id_to_word[1]] = word_to_id["<UNK>"]
label_to_id[id_to_label[0]] = label_to_id["O"]

label_to_id["O"] = 0
word_to_id["<PAD>"] = 0
word_to_id["<UNK>"] = 1

id_to_word = {word_to_id[word]: word for word in word_to_id}
id_to_label = {label_to_id[label]: label for label in label_to_id}

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
 def do_padding(sequences, length = MAX_LENGTH):
    return pad_sequences(sequences, maxlen = length)

train_sents_padded = do_padding(train_sents)
dev_sents_padded = do_padding(dev_sents)
test_sents_padded = do_padding(test_sents)

train_labels_padded = torch.LongTensor(do_padding(train_labels))
dev_labels_padded = do_padding(dev_labels)
test_labels_padded = do_padding(test_labels)

# TODO: if we want 1-hot vectors:
#train_labels_padded = to_categorical(do_padding(train_labels), NUM_LABELS)
#dev_labels_padded = to_categorical(do_padding(dev_labels), NUM_LABELS)
#test_labels_padded = to_categorical(do_padding(test_labels), NUM_LABELS)

In [ ]:
train_sents_padded

In [ ]:
#https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Sequence-Labeling/blob/master/dynamic_rnn.py

In [ ]:
import torch
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [ ]:
# Create an embedding layer, with 0 vectors for the pads
embeds = nn.Embedding(len(word_to_id), 50, padding_idx=0)

# Create an LSTM layer
lstm = nn.LSTM(50, 50, bidirectional=False, batch_first=True)

In [ ]:
seqs = torch.LongTensor(train_sents_padded)
embeddings = embeds(seqs)
out_static, _ = lstm(embeddings)
assert out_static.size(1) == embeddings.size(1)

In [ ]:
#seq_lens=...
#packed_seqs = pack_padded_sequence(embeddings, seq_lens.tolist(), batch_first=True)

In [ ]:
# https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Sequence-Labeling/blob/master/train.py
class LstmModel(nn.Module):
    def __init__(self, vocab_size, tagset_size, embedding_dim, hidden_dim, padding_idx=0):
        super(LstmModel, self).__init__()
        self.tagset_size = tagset_size
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        
        self.embeds = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=False, batch_first=True)
        self.emission = nn.Linear(hidden_dim, tagset_size)
    
    def forward(self, token_idxs):
        embeds = self.embeds(token_idxs)
        hidden, _ = self.lstm(embeds)
        pred = self.emission(hidden)
        return pred

In [ ]:
model = LstmModel(vocab_size = len(word_to_id), tagset_size = len(label_to_id), embedding_dim=30, hidden_dim=50)

In [ ]:
s_batch = torch.LongTensor(train_sents_padded[0:2])
s_batch

In [ ]:
out = model(s_batch)

In [ ]:
out[0].shape

In [ ]:
from torch.utils.data import Dataset

class SeqDataset(Dataset):
    def __init__(self, tokens, labels):
        self.tokens = tokens
        self.labels = labels
        self.data_size = len(self.tokens)

    def __getitem__(self, i):
        return self.tokens[i], self.labels[i]

    def __len__(self):
        return self.data_size

In [ ]:
def adjust_learning_rate(optimizer, new_lr):
    """
    Shrinks learning rate by a specified factor.
    :param optimizer: optimizer whose learning rates must be decayed
    :param new_lr: new learning rate
    """

    print("\nDECAYING learning rate.")
    for param_group in optimizer.param_groups:
        param_group['lr'] = new_lr
    print("The new learning rate is %f\n" % (optimizer.param_groups[0]['lr'],))

In [ ]:
# Training parameters
start_epoch = 0  # start at this epoch
batch_size = 10  # batch size
lr = 0.015  # learning rate
lr_decay = 0.05  # decay learning rate by this amount
momentum = 0.9  # momentum
workers = 1  # number of workers for loading data in the DataLoader
epochs = 2  # number of epochs to run without early-stopping
grad_clip = 5.  # clip gradients at this value
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_sents_padded.shape

In [ ]:
def clip_gradient(optimizer, grad_clip):
    """
    Clip gradients computed during backpropagation to prevent gradient explosion.
    :param optimizer: optimized with the gradients to be clipped
    :param grad_clip: gradient clip value
    """
    for group in optimizer.param_groups:
        for param in group['params']:
            if param.grad is not None:
                param.grad.data.clamp_(-grad_clip, grad_clip)

In [ ]:
optimizer = torch.optim.SGD(params=filter(lambda p: p.requires_grad, model.parameters()), lr=lr, momentum=momentum)

criterion = nn.CrossEntropyLoss().to(device)

train_loader = torch.utils.data.DataLoader(SeqDataset(train_sents_padded, train_labels_padded), batch_size=batch_size, shuffle=True,
                                               num_workers=workers, pin_memory=False)
val_loader = torch.utils.data.DataLoader(SeqDataset(dev_sents_padded, dev_labels_padded), batch_size=batch_size, shuffle=True,
                                             num_workers=workers, pin_memory=False)

# Epochs
for epoch in range(start_epoch, epochs):
    model.train()
    for i, (tokens, labels) in enumerate(train_loader):
    #for (tokens, labels) in train_loader:
        
#        print(tokens.shape)
#        print(s_batch.shape)
#        print(train_sents_padded.shape)
        
        #logits = model(s_batch)
        #print(logits.shape)

        # batchsize x nb_classes x seq_len
        logits = model(tokens).permute(0,2,1)
        #print(logits.shape)
        #print(labels.shape)
        
        #labels = torch.LongTensor(labels)
        
        
        loss = criterion(logits, labels)
                # Back prop.
        optimizer.zero_grad()
        loss.backward()

        if grad_clip is not None:
            clip_gradient(optimizer, grad_clip)

        optimizer.step()

    # One epoch's validation
    #val_f1 = validate(val_loader=val_loader,
    #                  model=model,
    #                  crf_criterion=crf_criterion,
    #                  vb_decoder=vb_decoder)
    adjust_learning_rate(optimizer, lr / (1 + (epoch + 1) * lr_decay))


In [ ]:



train_labels

In [ ]:
lf_to_newlabel = dict()

for oldlabel in label_to_id.keys():
    #print(label)
    #print(label.split(".")[0].split("-")[-1])
    newlabel = oldlabel.split(".")[0].split("-")[-1]
    lf_to_newlabel[oldlabel] = newlabel

del lf_to_newlabel["O"]

In [ ]:
lf_to_newlabel

In [ ]:
print(len(set(lf_to_newlabel.keys())))
print(len(set(lf_to_newlabel.values())))

In [ ]:
# TODO:
# 1. use original labels, without "O" to represent Z-matrix
# 2. make T matrix
# 3. do majority vote
# 4. make new label vector
# 5. train
# 6. check result of training

In [ ]:
# 1. use original labels, without "O" to represent Z-matrix
train_labels_padded = to_categorical(do_padding(train_labels), NUM_LABELS)[:,:,1:]
dev_labels_padded = to_categorical(do_padding(dev_labels), NUM_LABELS)[:,:,1:]
test_labels_padded = to_categorical(do_padding(test_labels), NUM_LABELS)[:,:,1:]

In [ ]:
sorted(list(id_to_label.items()))

In [ ]:
newlabel_to_id = dict()

for i, nl in enumerate(set(lf_to_newlabel.values())):
    newlabel_to_id[nl] = i

In [ ]:
lfid_to_nlid = dict()

In [ ]:
for lf_id in id_to_label:
    if lf_id == 0:
        continue
    lf_name = id_to_label[lf_id]
    newlabel_name = lf_to_newlabel[lf_name]
    nl_id = newlabel_to_id[newlabel_name]
    # "O" is not a lf in the 1-hot encoding.
    lfid_to_nlid[lf_id - 1] = nl_id

In [ ]:
lfid_to_nlid

In [ ]:
import numpy as np

t_matrix = np.zeros((len(lfid_to_nlid), len(newlabel_to_id)))

In [ ]:
for lfid in lfid_to_nlid:
    t_matrix[lfid, lfid_to_nlid[lfid]] = 1
    
t_matrix

In [ ]:
train_labels[0]

In [ ]:
print(train_labels_padded.shape)
print(t_matrix.shape)
# train_labels_padded <-> "Z matrix" <-> lf matches <-> original fine-grained 'labels'
print(train_labels_padded.dot(t_matrix).shape)
train_newlabels_padded = train_labels_padded.dot(t_matrix)

In [ ]:
train_newlabels_padded[0]

In [ ]:
has_label = train_newlabels_padded.max(axis=2)
has_label


In [ ]:
label_ids = (train_newlabels_padded.argmax(axis=2) + 1) * has_label - 1
label_ids

In [ ]:
id_to_newlabel = {i:l for l,i in newlabel_to_id.items()}

id_to_newlabel[-1] = "O"
id_to_newlabel

In [ ]:
nl1 = label_ids[0]
#[(id_to_word[wid], id_to_label[lid]) for (wid, lid) in zip(s1,nl1)]
nl1

In [ ]:
[id_to_newlabel[lid] for lid in nl1]